In [52]:
import os 
import json 
import zipfile
from rocrate.utils import *
from datetime import datetime

# Checking RO-Crate Metadata Root Data Entity

### RO-Crate Self Descriptor

In [26]:
def get_norm_value(json_entity, prop):
    """\
    Get a normalized value for a property (always as a list of strings).
    """
    value = json_entity.get(prop, [])
    if not isinstance(value, list):
        value = [value]
    try:
        return [_ if isinstance(_, str) else _["@id"] for _ in value]
    except (TypeError, KeyError):
        raise ValueError(f"Malformed value for {prop!r}: {json_entity.get(prop)!r}")

        
def self_descriptor_check(tar_file, extension):
    """\
    Check the self descriptor in RO-Crate
    Please check the requirements details in: 
    <https://www.researchobject.org/ro-crate/1.1/root-data-entity.html>
    """
    NAME = "Self descriptor check"
    error_message = "self descriptor is not provided"
    
    with open (os.path.join(tar_file, "ro-crate-metadata.json"), 'r') as file:
        metadata = json.load(file)
        graph = metadata['@graph']
    
    for entity in graph:
        type = get_norm_value(entity, "@type")
        if type == ['CreativeWork']:
            if (get_norm_value(entity, "@id") == ['ro-crate-metadata.json'] or get_norm_value(entity, "@id") == ['ro-crate-metadata.jsonld']) and get_norm_value(entity, "about") == ['./']:
                link_ = get_norm_value(entity, "conformsTo")[0]
                if link_.startswith("https://w3id.org/ro/crate/"): 
                    return NAME, True
    return NAME, error_message, False

### Direct property of Root Data Entity

In [55]:
def datetime_valid(dt_str):
    try: 
        datetime.fromisoformat(dt_str)
    except: 
        return False
    return True


def direct_property_check(tar_file, extension):
    """\
    A valid RO-Crate MUST meets the direct property requirements
    Please check the requirements details in: 
    <https://www.researchobject.org/ro-crate/1.1/root-data-entity.html>
    """
    NAME = "Direct property check"
    error_message = ["datePublished is not in ISO 8601 date format", "Directory property of RO-Crate is wrong"]
    
    with open (os.path.join(tar_file, "ro-crate-metadata.json"), 'r') as file:
        metadata = json.load(file)
        graph = metadata['@graph']
        
    for entity in graph: 
        type = get_norm_value(entity, '@type')
        if type[0] == 'Dataset' and get_norm_value(entity, '@id')[0].endswith('/'):
            for _ in get_norm_value(entity, 'datePublished'):
                if datetime_valid(_) == True:
                    return NAME, datetime_valid(_)
                else: 
                    return NAME, error_message[0], datetime_valid(_)
    return NAME, error_message[1], False
                

# Checking RO-Crate Data Entity

### Referencing file or folder from root data entity

In [56]:

def Referencing_check(tar_file, extension):
    """/
    Where file or folder are represented as Data Entity in RO-Crate JSON-LD
    There MUST be linked to, directly or indirectly, hasPart in Root Data Entity.
    For more information, please check : 
    <https://www.researchobject.org/ro-crate/1.1/root-data-entity.html>
    """
    
    NAME = "referencing check"
    error_message = "The referencing is wrong"
    
    with open("ro-crate-metadata.json", w) as file:
        metadata = file.load()
        graph = metadata['@graph']
        
    for entity in graph: 
        
    
    